# Языковые модели для классификации

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import keras.layers as layers

from collections import Counter
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Input, Embedding, BatchNormalization, LSTM, Dense, Concatenate
from keras.models import Model

from keras.utils import plot_model


Загружаем IMDB dataset

In [0]:
from sklearn.model_selection import train_test_split


# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
  
    for file_path in os.listdir(directory):
        with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
          data["sentence"].append(f.read())
          data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
     
    return pd.DataFrame.from_dict(data)


# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)


# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
        fname="aclImdb.tar.gz", 
        origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
        extract=True)
 
    train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
    return train_df, test_df


# Load and process the dataset files from local storage.
def download_and_load_datasets_local(force_download=False):
  
    train_df = load_dataset(os.path.join(os.path.dirname("./dataset/"), 
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname("./dataset/"), 
                                      "aclImdb", "test"))
    train_df.append(test_df, ignore_index=True)
    
    train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=42)
  
    return train_df, test_df

In [4]:
train_df, test_df = download_and_load_datasets()
print(train_df.head())

84131840/84125825 [==============================] - 8s 0us/step
                                            sentence sentiment  polarity
0  The Bible teaches us that the love of money is...        10         1
1  *I mark where there are spoilers! Overall comm...        10         1
2  ...let me count the ways.<br /><br />1. A titl...         1         0
3  Despite being quite far removed from my expect...         8         1
4  This is a very entertaining flick, considering...         9         1


### Построим словарь

In [0]:
# parameter of max word length
time_steps = 100


# building vocabulary from dataset
def build_vocabulary(sentence_list):
    unique_words = " ".join(sentence_list).strip().split()
    word_count = Counter(unique_words).most_common()
    vocabulary = {}
    for word, _ in word_count:
        vocabulary[word] = len(vocabulary)        

    return vocabulary


# Get vocabulary vectors from document list
# Vocabulary vector, Unknown word is 1 and padding is 0
# INPUT: raw sentence list
# OUTPUT: vocabulary vectors list
def get_voc_vec(document_list, vocabulary):    
    voc_ind_sentence_list = []
    for document in document_list:
        voc_idx_sentence = []
        word_list = document.split()
        
        for w in range(time_steps):
            if w < len(word_list):
                # pickup vocabulary id and convert unknown word into 1
                voc_idx_sentence.append(vocabulary.get(word_list[w], -1) + 2)
            else:
                # padding with 0
                voc_idx_sentence.append(0)
            
        voc_ind_sentence_list.append(voc_idx_sentence)
        
    return np.array(voc_ind_sentence_list)


vocabulary = build_vocabulary(train_df["sentence"])


### Создаем ELMo модель для классификации
Загружаем модель с tensorflow hub

In [0]:
# Reduce TensorFlow logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

# Instantiate the elmo model
elmo_module = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

# Initialize session
sess = tf.Session()
K.set_session(sess)

K.set_learning_phase(1)

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

### Создаем генератор батчей

In [0]:
# mini-batch generator
def batch_iter(data, labels, batch_size, shuffle=True):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    print("batch_size", batch_size)
    print("num_batches_per_epoch", num_batches_per_epoch)

    def data_generator():
        data_size = len(data)

        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                                
                X_voc = get_voc_vec(shuffled_data[start_index: end_index], vocabulary)
                                
                sentence_split_list = []
                sentence_split_length_list = []
            
                for sentence in shuffled_data[start_index: end_index]:    
                    sentence_split = sentence.split()
                    sentence_split_length = len(sentence_split)
                    sentence_split += ["NaN"] * (time_steps - sentence_split_length)
                    
                    sentence_split_list.append((" ").join(sentence_split))
                    sentence_split_length_list.append(sentence_split_length)
        
                X_elmo = np.array(sentence_split_list)

                X = [X_voc, X_elmo]
                y = shuffled_labels[start_index: end_index]
                
                yield X, y

    return num_batches_per_epoch, data_generator()


### Задаем архитектуру модели

In [0]:
# embed elmo method
def make_elmo_embedding(x):
    embeddings = elmo_module(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["elmo"]
    
    return embeddings


In [9]:
# elmo embedding dimension
elmo_dim = 1024

# Input Layers
word_input = Input(shape=(None, ), dtype='int32')  # (batch_size, sent_length)
elmo_input = Input(shape=(None, ), dtype=tf.string)  # (batch_size, sent_length, elmo_size)

# Hidden Layers
word_embedding = Embedding(input_dim=len(vocabulary), output_dim=128, mask_zero=True)(word_input)
elmo_embedding = layers.Lambda(make_elmo_embedding, output_shape=(None, elmo_dim))(elmo_input)
word_embedding = Concatenate()([word_embedding, elmo_embedding])
word_embedding = BatchNormalization()(word_embedding)
x = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(word_embedding)

# Output Layer
predict = Dense(units=1, activation='sigmoid')(x)


model = Model(inputs=[word_input, elmo_input], outputs=predict)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    35918976    input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 1024)   0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenat

### Создадим данные

In [0]:
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:time_steps]) for t in train_text]
train_text = np.array(train_text)
train_label = np.array(train_df['polarity'].tolist())

test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:time_steps]) for t in test_text]
test_text = np.array(test_text)
test_label = np.array(test_df['polarity'].tolist())

In [11]:
# mini-batch size
batch_size = 32

train_steps, train_batches = batch_iter(train_text,
                                        np.array(train_df["polarity"]),
                                        batch_size)
valid_steps, valid_batches = batch_iter(test_text,
                                        np.array(test_df["polarity"]),
                                        batch_size)

batch_size 32
num_batches_per_epoch 782
batch_size 32
num_batches_per_epoch 782


In [0]:
history = model.fit_generator(train_batches, train_steps,
                              epochs=5,
                              validation_data=valid_batches,
                              validation_steps=valid_steps,
                              callbacks=[tb_cb])

Epoch 1/5
715/782 [==========================>...] - ETA: 2:31 - loss: 0.5188 - acc: 0.7451

In [0]:
score = model.evaluate_generator(valid_batches, valid_steps)
print('Test score:', score[0])
print('Test accuracy;', score[1])


Test score: 1.0284352650165558
Test accuracy; 0.77596
